In [1]:
import pandas as pd
import numpy as np

# Load the DataFrame from the CSV file
df = pd.read_csv('upsampled_np_arryed_images.csv')
df.drop(columns=['image_path', 'attributes', 'processed_image'], inplace=True)

# Load images and attributes
# class_names = np.load("class_names.npy", allow_pickle=True).item()
# class_attributes = 
# class_labels = list(class_names.values())

# df['attributes'] = np.load("upsampled_attributes.npy", allow_pickle=True)
df['image_array'] = np.load("processed_images.npy", allow_pickle=True)

In [2]:
# Prepare the X
X = np.stack(df['image_array'].to_numpy(),axis=0)

# Efficientnet Test

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
y = np.stack(df['label'],axis=0)
y = np.array(y).reshape(-1, 1)  # Reshape to a 2D array

encoder = OneHotEncoder(sparse=False, categories='auto')
y_one_hot = encoder.fit_transform(y)
y_one_hot = y_one_hot.reshape(y_one_hot.shape[0], -1)

print(y_one_hot.shape)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

print(y_train.shape)

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras import regularizers
from tensorflow.keras.utils import to_categorical

efnet = tf.keras.applications.efficientnet.EfficientNetB3(include_top=False, weights="imagenet", input_shape=(250, 250, 3), pooling='max')

model = Sequential([
    efnet,
    BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001),
    Dense(256, kernel_regularizer=regularizers.l2(0.016), activity_regularizer=regularizers.l1(0.006), bias_regularizer=regularizers.l1(0.006), activation='relu'),
    Dropout(rate=0.45, seed=42),
    Dense(200, activation='softmax') 
])

model.compile(Adamax(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb3 (Functional)     │ ?                      │    10,783,535 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,783,535 (41.14 MB)

 Trainable params: 10,696,232 (40.80 MB)

 Non-trainable params: 87,303 (341.03 KB)

In [ ]:
history = model.fit(x= X_train, y= y_train, epochs= 15,
                    validation_data= (X_test, y_test), validation_steps= None, shuffle= False)

In [6]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

44/44 ━━━━━━━━━━━━━━━━━━━━ 29s 641ms/step - accuracy: 0.0014 - loss: 5.5833
Test Loss: 5.582966327667236, Test Accuracy: 0.002142857061699033


In [7]:
prediction = model.predict(X_test)

44/44 ━━━━━━━━━━━━━━━━━━━━ 31s 653ms/step


In [9]:
model.save('efnet.keras')

In [3]:
# Round all values in the 'attributes' column to 6 decimal places for small model training
df['attributes'] = df['attributes'].apply(lambda x: np.round(x, 6))

# Bill Shape

In [ ]:
# Keep only the relevant values in each numpy array in the 'attributes' column
df['attributes_train'] = df['attributes'].apply(lambda x: x[0:9])

# Check the DataFrame to ensure the values have been updated
df['attributes_train'][0]


In [ ]:
# Prepare the y
y = np.stack(df['attributes_train'].to_numpy(),axis=0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
length_of_output = y_test.shape[1]

In [ ]:
# Define the CNN model
inputs = Input(shape=(250, 250, 3))
x = Conv2D(16, (3, 3), activation='relu')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(length_of_output)(x)  # Output layer with length_of_output units for the attributes

model = Model(inputs=inputs, outputs=outputs)

# Compile the model with mse as the loss function
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])


In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {accuracy}')

In [ ]:
# Save the knowledge
model.save('m1_bill_shape.keras')

# Wing Color

In [4]:
# Keep only the appropriate values in each numpy array in the 'attributes' column
df['attributes_train'] = df['attributes'].apply(lambda x: x[9:24])

In [5]:
# Prepare the y
y = np.stack(df['attributes_train'].to_numpy(),axis=0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

length_of_output = y_test.shape[1]

In [6]:
# Define the CNN model - no color filter
inputs = Input(shape=(250, 250, 3))
x = Conv2D(16, (3, 3), activation='relu')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
outputs = Dense(length_of_output, activation='sigmoid')(x)  # Output layer with length_of_output units for the attributes

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Summary of the model
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 250, 250, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 248, 248, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 124, 124, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 122, 122, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 61, 61, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 59, 59, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 15)             │       807,375 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 830,959 (3.17 MB)

 Trainable params: 830,959 (3.17 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Define the CNN model - color filter
inputs = Input(shape=(250, 250, 3))

# Initial layer to focus on color
color_layer = Conv2D(16, (1, 1), activation='relu')(inputs)

# Continue with the rest of the model
x = Conv2D(32, (3, 3), activation='relu')(color_layer)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
outputs = Dense(length_of_output, activation='sigmoid')(x)  # Output layer with length_of_output units for the attributes

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Summary of the model
model.summary()


In [7]:
# Train the model
history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {accuracy}')

Epoch 1/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 29s 150ms/step - accuracy: 0.2916 - loss: 0.0108 - val_accuracy: 0.3064 - val_loss: 0.0035
Epoch 2/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 100ms/step - accuracy: 0.3123 - loss: 0.0035 - val_accuracy: 0.3064 - val_loss: 0.0035
Epoch 3/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 101ms/step - accuracy: 0.3089 - loss: 0.0035 - val_accuracy: 0.3064 - val_loss: 0.0035
Epoch 4/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 101ms/step - accuracy: 0.3057 - loss: 0.0035 - val_accuracy: 0.3064 - val_loss: 0.0035
Epoch 5/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 102ms/step - accuracy: 0.3093 - loss: 0.0035 - val_accuracy: 0.3064 - val_loss: 0.0035
Epoch 6/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 100ms/step - accuracy: 0.3071 - loss: 0.0035 - val_accuracy: 0.3064 - val_loss: 0.0035
Epoch 7/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 17s 100ms/step - accuracy: 0.3078 - loss: 0.0035 - val_accuracy: 0.3064 - val_loss: 0.0035
Epoch 8/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 101ms/step - accuracy: 0.3188 - loss: 0

In [8]:
# Save the knowledge
model.save('m2_wing_color.keras')

# Upperparts Color

In [4]:
# Keep only the appropriate values in each numpy array in the 'attributes' column
df['attributes_train'] = df['attributes'].apply(lambda x: x[24:39])

# Prepare the y
y = np.stack(df['attributes_train'].to_numpy(),axis=0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

length_of_output = y_test.shape[1]

In [5]:
# Define the CNN model - color filter
inputs = Input(shape=(250, 250, 3))

# Initial layer to focus on color
color_layer = Conv2D(16, (1, 1), activation='relu')(inputs)

# Continue with the rest of the model
x = Conv2D(32, (3, 3), activation='relu')(color_layer)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
outputs = Dense(length_of_output, activation='softmax')(x)  # Output layer with length_of_output units for the attributes

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Summary of the model
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 250, 250, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 250, 250, 16)   │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 248, 248, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 124, 124, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 122, 122, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 61, 61, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 238144)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 238144)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 15)             │     3,572,175 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,595,375 (13.72 MB)

 Trainable params: 3,595,375 (13.72 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# Train the model
history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {accuracy}')

Epoch 1/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 66s 360ms/step - accuracy: 0.2512 - loss: 0.0061 - val_accuracy: 0.2986 - val_loss: 0.0028
Epoch 2/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 58s 330ms/step - accuracy: 0.2905 - loss: 0.0028 - val_accuracy: 0.2986 - val_loss: 0.0028
Epoch 3/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 60s 340ms/step - accuracy: 0.3053 - loss: 0.0027 - val_accuracy: 0.2979 - val_loss: 0.0028
Epoch 4/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 58s 329ms/step - accuracy: 0.3350 - loss: 0.0027 - val_accuracy: 0.2957 - val_loss: 0.0028
Epoch 5/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 57s 324ms/step - accuracy: 0.3968 - loss: 0.0025 - val_accuracy: 0.2886 - val_loss: 0.0029
Epoch 6/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 57s 326ms/step - accuracy: 0.4825 - loss: 0.0023 - val_accuracy: 0.2843 - val_loss: 0.0029
Epoch 7/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 57s 325ms/step - accuracy: 0.5382 - loss: 0.0022 - val_accuracy: 0.2914 - val_loss: 0.0029
Epoch 8/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 58s 329ms/step - accuracy: 0.5682 - loss: 0

In [7]:
# Save the knowledge
model.save('m3_upperparts_color.keras')

# Underparts Color

In [8]:
# Keep only the appropriate values in each numpy array in the 'attributes' column
df['attributes_train'] = df['attributes'].apply(lambda x: x[39:54])

# Prepare the y
y = np.stack(df['attributes_train'].to_numpy(),axis=0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

length_of_output = y_test.shape[1]

In [9]:
# Define the CNN model - color filter
inputs = Input(shape=(250, 250, 3))

# Initial layer to focus on color
color_layer = Conv2D(16, (1, 1), activation='relu')(inputs)

# Continue with the rest of the model
x = Conv2D(32, (3, 3), activation='relu')(color_layer)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
outputs = Dense(length_of_output, activation='softmax')(x)  # Output layer with length_of_output units for the attributes

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 250, 250, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 250, 250, 16)   │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 248, 248, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 124, 124, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 122, 122, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 61, 61, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 238144)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 238144)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │     3,572,175 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,595,375 (13.72 MB)

 Trainable params: 3,595,375 (13.72 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
# Train the model
history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {accuracy}')

Epoch 1/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 69s 369ms/step - accuracy: 0.3237 - loss: 0.0073 - val_accuracy: 0.4200 - val_loss: 0.0034
Epoch 2/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 58s 330ms/step - accuracy: 0.4392 - loss: 0.0033 - val_accuracy: 0.4200 - val_loss: 0.0034
Epoch 3/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 57s 328ms/step - accuracy: 0.4370 - loss: 0.0033 - val_accuracy: 0.4200 - val_loss: 0.0034
Epoch 4/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 58s 329ms/step - accuracy: 0.4595 - loss: 0.0032 - val_accuracy: 0.4164 - val_loss: 0.0035
Epoch 5/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 58s 330ms/step - accuracy: 0.5106 - loss: 0.0030 - val_accuracy: 0.4021 - val_loss: 0.0035
Epoch 6/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 58s 332ms/step - accuracy: 0.5995 - loss: 0.0028 - val_accuracy: 0.3793 - val_loss: 0.0035
Epoch 7/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 57s 328ms/step - accuracy: 0.6431 - loss: 0.0027 - val_accuracy: 0.4043 - val_loss: 0.0035
Epoch 8/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 57s 328ms/step - accuracy: 0.6612 - loss: 0

In [11]:
# Save the knowledge
model.save('m4_underparts_color.keras')

# Breast Pattern

In [12]:
# Keep only the appropriate values in each numpy array in the 'attributes' column
df['attributes_train'] = df['attributes'].apply(lambda x: x[54:58])

# Prepare the y
y = np.stack(df['attributes_train'].to_numpy(),axis=0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

length_of_output = y_test.shape[1]

In [13]:
# Define the CNN model - no color filter
inputs = Input(shape=(250, 250, 3))
x = Conv2D(16, (3, 3), activation='relu')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
outputs = Dense(length_of_output, activation='sigmoid')(x)  # Output layer with length_of_output units for the attributes

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Summary of the model
model.summary()


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 250, 250, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 248, 248, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 124, 124, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 122, 122, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 61, 61, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 59, 59, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │       215,300 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 238,884 (933.14 KB)

 Trainable params: 238,884 (933.14 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# Train the model
history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {accuracy}')

Epoch 1/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 33s 162ms/step - accuracy: 0.3728 - loss: 0.0148 - val_accuracy: 0.7543 - val_loss: 0.0045
Epoch 2/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 106ms/step - accuracy: 0.7363 - loss: 0.0027 - val_accuracy: 0.7543 - val_loss: 0.0021
Epoch 3/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 102ms/step - accuracy: 0.7441 - loss: 0.0022 - val_accuracy: 0.7543 - val_loss: 0.0021
Epoch 4/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 103ms/step - accuracy: 0.7440 - loss: 0.0021 - val_accuracy: 0.7543 - val_loss: 0.0021
Epoch 5/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 17s 98ms/step - accuracy: 0.7555 - loss: 0.0021 - val_accuracy: 0.7543 - val_loss: 0.0021
Epoch 6/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 101ms/step - accuracy: 0.7519 - loss: 0.0020 - val_accuracy: 0.7579 - val_loss: 0.0021
Epoch 7/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 104ms/step - accuracy: 0.7543 - loss: 0.0020 - val_accuracy: 0.7557 - val_loss: 0.0021
Epoch 8/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 104ms/step - accuracy: 0.7534 - loss: 0.

In [15]:
# Save the knowledge
model.save('m5_breast_pattern.keras')

# Back Color

In [16]:
# Keep only the appropriate values in each numpy array in the 'attributes' column
df['attributes_train'] = df['attributes'].apply(lambda x: x[58:73])

# Prepare the y
y = np.stack(df['attributes_train'].to_numpy(),axis=0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

length_of_output = y_test.shape[1]

In [17]:
# Define the CNN model - color filter
inputs = Input(shape=(250, 250, 3))

# Initial layer to focus on color
color_layer = Conv2D(16, (1, 1), activation='relu')(inputs)

# Continue with the rest of the model
x = Conv2D(32, (3, 3), activation='relu')(color_layer)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
outputs = Dense(length_of_output, activation='softmax')(x)  # Output layer with length_of_output units for the attributes

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 250, 250, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 250, 250, 16)   │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 248, 248, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 124, 124, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 122, 122, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 61, 61, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 238144)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 238144)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 15)             │     3,572,175 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,595,375 (13.72 MB)

 Trainable params: 3,595,375 (13.72 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# Train the model
history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {accuracy}')

Epoch 1/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 72s 380ms/step - accuracy: 0.2199 - loss: 0.0046 - val_accuracy: 0.2693 - val_loss: 0.0031
Epoch 2/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 58s 331ms/step - accuracy: 0.2705 - loss: 0.0030 - val_accuracy: 0.2621 - val_loss: 0.0030
Epoch 3/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 57s 327ms/step - accuracy: 0.2964 - loss: 0.0030 - val_accuracy: 0.2657 - val_loss: 0.0030
Epoch 4/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 57s 326ms/step - accuracy: 0.3552 - loss: 0.0029 - val_accuracy: 0.2693 - val_loss: 0.0030
Epoch 5/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 57s 325ms/step - accuracy: 0.4144 - loss: 0.0028 - val_accuracy: 0.2750 - val_loss: 0.0031
Epoch 6/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 58s 330ms/step - accuracy: 0.4900 - loss: 0.0026 - val_accuracy: 0.2729 - val_loss: 0.0031
Epoch 7/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 58s 329ms/step - accuracy: 0.5513 - loss: 0.0025 - val_accuracy: 0.2714 - val_loss: 0.0032
Epoch 8/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 57s 326ms/step - accuracy: 0.5908 - loss: 0

In [19]:
# Save the knowledge
model.save('m6_back_color.keras')

# Tail Shape

In [20]:
# Keep only the appropriate values in each numpy array in the 'attributes' column
df['attributes_train'] = df['attributes'].apply(lambda x: x[73:79])

# Prepare the y
y = np.stack(df['attributes_train'].to_numpy(),axis=0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

length_of_output = y_test.shape[1]

In [21]:
# Define the CNN model - no color filter
inputs = Input(shape=(250, 250, 3))
x = Conv2D(16, (3, 3), activation='relu')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
outputs = Dense(length_of_output, activation='sigmoid')(x)  # Output layer with length_of_output units for the attributes

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Summary of the model
model.summary()


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 250, 250, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 248, 248, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 124, 124, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 122, 122, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 61, 61, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 59, 59, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 6)              │       322,950 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 346,534 (1.32 MB)

 Trainable params: 346,534 (1.32 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
# Train the model
history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {accuracy}')

Epoch 1/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 32s 159ms/step - accuracy: 0.5960 - loss: 0.0096 - val_accuracy: 0.6407 - val_loss: 0.0030
Epoch 2/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 106ms/step - accuracy: 0.6303 - loss: 0.0030 - val_accuracy: 0.6407 - val_loss: 0.0030
Epoch 3/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 104ms/step - accuracy: 0.6451 - loss: 0.0030 - val_accuracy: 0.6407 - val_loss: 0.0030
Epoch 4/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 17s 97ms/step - accuracy: 0.6353 - loss: 0.0030 - val_accuracy: 0.6407 - val_loss: 0.0030
Epoch 5/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 100ms/step - accuracy: 0.6321 - loss: 0.0030 - val_accuracy: 0.6407 - val_loss: 0.0030
Epoch 6/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 17s 98ms/step - accuracy: 0.6457 - loss: 0.0030 - val_accuracy: 0.6407 - val_loss: 0.0030
Epoch 7/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 102ms/step - accuracy: 0.6237 - loss: 0.0030 - val_accuracy: 0.6407 - val_loss: 0.0030
Epoch 8/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 104ms/step - accuracy: 0.6405 - loss: 0.0

In [23]:
# Save the knowledge
model.save('m7_tail_shape.keras')

# Upper Tail Color

In [24]:
# Keep only the appropriate values in each numpy array in the 'attributes' column
df['attributes_train'] = df['attributes'].apply(lambda x: x[79:94])

# Prepare the y
y = np.stack(df['attributes_train'].to_numpy(),axis=0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

length_of_output = y_test.shape[1]

In [25]:
# Define the CNN model - no color filter
inputs = Input(shape=(250, 250, 3))
x = Conv2D(16, (3, 3), activation='relu')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
outputs = Dense(length_of_output, activation='sigmoid')(x)  # Output layer with length_of_output units for the attributes

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Summary of the model
model.summary()


Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 250, 250, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 248, 248, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 124, 124, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 122, 122, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 61, 61, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 59, 59, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 15)             │       807,375 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 830,959 (3.17 MB)

 Trainable params: 830,959 (3.17 MB)

 Non-trainable params: 0 (0.00 B)

In [26]:
# Train the model
history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {accuracy}')

Epoch 1/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 31s 151ms/step - accuracy: 0.2828 - loss: 0.0109 - val_accuracy: 0.3157 - val_loss: 0.0025
Epoch 2/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 106ms/step - accuracy: 0.3034 - loss: 0.0025 - val_accuracy: 0.3157 - val_loss: 0.0025
Epoch 3/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 104ms/step - accuracy: 0.2878 - loss: 0.0025 - val_accuracy: 0.3157 - val_loss: 0.0025
Epoch 4/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 104ms/step - accuracy: 0.2994 - loss: 0.0025 - val_accuracy: 0.3157 - val_loss: 0.0025
Epoch 5/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 103ms/step - accuracy: 0.3086 - loss: 0.0025 - val_accuracy: 0.3157 - val_loss: 0.0025
Epoch 6/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 103ms/step - accuracy: 0.2913 - loss: 0.0025 - val_accuracy: 0.3157 - val_loss: 0.0025
Epoch 7/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 104ms/step - accuracy: 0.2966 - loss: 0.0025 - val_accuracy: 0.3157 - val_loss: 0.0025
Epoch 8/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 104ms/step - accuracy: 0.2984 - loss: 0

In [27]:
# Save the knowledge
model.save('m8_upper_tail_color.keras')

# Head Pattern

In [28]:
# Keep only the appropriate values in each numpy array in the 'attributes' column
df['attributes_train'] = df['attributes'].apply(lambda x: x[94:105])

# Prepare the y
y = np.stack(df['attributes_train'].to_numpy(),axis=0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

length_of_output = y_test.shape[1]

In [29]:
# Define the CNN model - no color filter
inputs = Input(shape=(250, 250, 3))
x = Conv2D(16, (3, 3), activation='relu')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
outputs = Dense(length_of_output, activation='sigmoid')(x)  # Output layer with length_of_output units for the attributes

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Summary of the model
model.summary()


Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 250, 250, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 248, 248, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 124, 124, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 122, 122, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 61, 61, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 59, 59, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 11)             │       592,075 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 615,659 (2.35 MB)

 Trainable params: 615,659 (2.35 MB)

 Non-trainable params: 0 (0.00 B)

In [30]:
# Train the model
history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {accuracy}')

Epoch 1/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 33s 171ms/step - accuracy: 0.2413 - loss: 0.0091 - val_accuracy: 0.3836 - val_loss: 0.0018
Epoch 2/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 106ms/step - accuracy: 0.2934 - loss: 0.0018 - val_accuracy: 0.3836 - val_loss: 0.0018
Epoch 3/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 103ms/step - accuracy: 0.2881 - loss: 0.0018 - val_accuracy: 0.3836 - val_loss: 0.0018
Epoch 4/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 105ms/step - accuracy: 0.2973 - loss: 0.0018 - val_accuracy: 0.3836 - val_loss: 0.0018
Epoch 5/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 106ms/step - accuracy: 0.2950 - loss: 0.0018 - val_accuracy: 0.3836 - val_loss: 0.0018
Epoch 6/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 103ms/step - accuracy: 0.2926 - loss: 0.0018 - val_accuracy: 0.3836 - val_loss: 0.0018
Epoch 7/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 104ms/step - accuracy: 0.3184 - loss: 0.0018 - val_accuracy: 0.3836 - val_loss: 0.0018
Epoch 8/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 104ms/step - accuracy: 0.3000 - loss: 0

In [32]:
# Save the knowledge
model.save('m9_head_pattern.keras')

# Breast Color

In [33]:
# Keep only the appropriate values in each numpy array in the 'attributes' column
df['attributes_train'] = df['attributes'].apply(lambda x: x[105:120])

# Prepare the y
y = np.stack(df['attributes_train'].to_numpy(),axis=0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

length_of_output = y_test.shape[1]

In [34]:
# Define the CNN model - no color filter
inputs = Input(shape=(250, 250, 3))
x = Conv2D(16, (3, 3), activation='relu')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
outputs = Dense(length_of_output, activation='sigmoid')(x)  # Output layer with length_of_output units for the attributes

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "functional_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)      │ (None, 250, 250, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_21 (Conv2D)              │ (None, 248, 248, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 124, 124, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (None, 122, 122, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 61, 61, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 59, 59, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_20 (MaxPooling2D) │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 15)             │       807,375 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 830,959 (3.17 MB)

 Trainable params: 830,959 (3.17 MB)

 Non-trainable params: 0 (0.00 B)

In [35]:
# Train the model
history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {accuracy}')

Epoch 1/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 32s 162ms/step - accuracy: 0.3333 - loss: 0.0107 - val_accuracy: 0.3236 - val_loss: 0.0028
Epoch 2/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 108ms/step - accuracy: 0.3284 - loss: 0.0028 - val_accuracy: 0.3236 - val_loss: 0.0028
Epoch 3/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 110ms/step - accuracy: 0.3454 - loss: 0.0028 - val_accuracy: 0.3236 - val_loss: 0.0028
Epoch 4/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 111ms/step - accuracy: 0.3423 - loss: 0.0028 - val_accuracy: 0.3236 - val_loss: 0.0028
Epoch 5/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 109ms/step - accuracy: 0.3321 - loss: 0.0028 - val_accuracy: 0.3236 - val_loss: 0.0028
Epoch 6/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 110ms/step - accuracy: 0.3280 - loss: 0.0028 - val_accuracy: 0.3236 - val_loss: 0.0028
Epoch 7/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 107ms/step - accuracy: 0.3381 - loss: 0.0028 - val_accuracy: 0.3236 - val_loss: 0.0028
Epoch 8/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 110ms/step - accuracy: 0.3337 - loss: 0

In [36]:
# Save the knowledge
model.save('m10_breast_color.keras')

# Throat Color

In [37]:
# Keep only the appropriate values in each numpy array in the 'attributes' column
df['attributes_train'] = df['attributes'].apply(lambda x: x[120:135])

# Prepare the y
y = np.stack(df['attributes_train'].to_numpy(),axis=0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

length_of_output = y_test.shape[1]

In [38]:
# Define the CNN model - no color filter
inputs = Input(shape=(250, 250, 3))
x = Conv2D(16, (3, 3), activation='relu')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
outputs = Dense(length_of_output, activation='sigmoid')(x)  # Output layer with length_of_output units for the attributes

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "functional_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 250, 250, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_24 (Conv2D)              │ (None, 248, 248, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_21 (MaxPooling2D) │ (None, 124, 124, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_25 (Conv2D)              │ (None, 122, 122, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_22 (MaxPooling2D) │ (None, 61, 61, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_26 (Conv2D)              │ (None, 59, 59, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_23 (MaxPooling2D) │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_8 (Flatten)             │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 15)             │       807,375 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 830,959 (3.17 MB)

 Trainable params: 830,959 (3.17 MB)

 Non-trainable params: 0 (0.00 B)

In [39]:
# Train the model
history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {accuracy}')

Epoch 1/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 31s 156ms/step - accuracy: 0.3600 - loss: 0.0102 - val_accuracy: 0.3871 - val_loss: 0.0025
Epoch 2/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 109ms/step - accuracy: 0.3870 - loss: 0.0025 - val_accuracy: 0.3871 - val_loss: 0.0025
Epoch 3/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 109ms/step - accuracy: 0.3945 - loss: 0.0025 - val_accuracy: 0.3871 - val_loss: 0.0025
Epoch 4/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 104ms/step - accuracy: 0.3952 - loss: 0.0025 - val_accuracy: 0.3871 - val_loss: 0.0025
Epoch 5/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 105ms/step - accuracy: 0.3989 - loss: 0.0025 - val_accuracy: 0.3871 - val_loss: 0.0025
Epoch 6/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 105ms/step - accuracy: 0.3965 - loss: 0.0025 - val_accuracy: 0.3871 - val_loss: 0.0025
Epoch 7/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 104ms/step - accuracy: 0.3925 - loss: 0.0025 - val_accuracy: 0.3871 - val_loss: 0.0025
Epoch 8/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 104ms/step - accuracy: 0.3931 - loss: 0

In [40]:
# Save the knowledge
model.save('m11_throat_color.keras')

# Eye Color

In [41]:
# Keep only the appropriate values in each numpy array in the 'attributes' column
df['attributes_train'] = df['attributes'].apply(lambda x: x[135:149])

# Prepare the y
y = np.stack(df['attributes_train'].to_numpy(),axis=0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

length_of_output = y_test.shape[1]

In [42]:
# Define the CNN model - color filter
inputs = Input(shape=(250, 250, 3))

# Initial layer to focus on color
color_layer = Conv2D(16, (1, 1), activation='relu')(inputs)

# Continue with the rest of the model
x = Conv2D(32, (3, 3), activation='relu')(color_layer)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
outputs = Dense(length_of_output, activation='softmax')(x)  # Output layer with length_of_output units for the attributes

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "functional_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)      │ (None, 250, 250, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_27 (Conv2D)              │ (None, 250, 250, 16)   │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_28 (Conv2D)              │ (None, 248, 248, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_24 (MaxPooling2D) │ (None, 124, 124, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_29 (Conv2D)              │ (None, 122, 122, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_25 (MaxPooling2D) │ (None, 61, 61, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_9 (Flatten)             │ (None, 238144)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 238144)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 14)             │     3,334,030 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,357,230 (12.81 MB)

 Trainable params: 3,357,230 (12.81 MB)

 Non-trainable params: 0 (0.00 B)

In [43]:
# Train the model
history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {accuracy}')

Epoch 1/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 73s 391ms/step - accuracy: 0.8785 - loss: 0.0111 - val_accuracy: 0.9607 - val_loss: 0.0030
Epoch 2/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 61s 346ms/step - accuracy: 0.9520 - loss: 0.0031 - val_accuracy: 0.9607 - val_loss: 0.0031
Epoch 3/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 59s 335ms/step - accuracy: 0.9537 - loss: 0.0031 - val_accuracy: 0.9607 - val_loss: 0.0030
Epoch 4/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 60s 341ms/step - accuracy: 0.9557 - loss: 0.0030 - val_accuracy: 0.9607 - val_loss: 0.0030
Epoch 5/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 60s 344ms/step - accuracy: 0.9518 - loss: 0.0031 - val_accuracy: 0.9607 - val_loss: 0.0030
Epoch 6/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 61s 347ms/step - accuracy: 0.9527 - loss: 0.0030 - val_accuracy: 0.9607 - val_loss: 0.0030
Epoch 7/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 61s 346ms/step - accuracy: 0.9509 - loss: 0.0030 - val_accuracy: 0.9607 - val_loss: 0.0030
Epoch 8/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 59s 339ms/step - accuracy: 0.9493 - loss: 0

In [44]:
# Save the knowledge
model.save('m12_eye_color.keras')

# Bill Length to Head

In [45]:
# Keep only the appropriate values in each numpy array in the 'attributes' column
df['attributes_train'] = df['attributes'].apply(lambda x: x[149:152])

# Prepare the y
y = np.stack(df['attributes_train'].to_numpy(),axis=0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

length_of_output = y_test.shape[1]

In [46]:
# Define the CNN model - no color filter
inputs = Input(shape=(250, 250, 3))
x = Conv2D(16, (3, 3), activation='relu')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
outputs = Dense(length_of_output, activation='sigmoid')(x)  # Output layer with length_of_output units for the attributes

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "functional_21"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_10 (InputLayer)     │ (None, 250, 250, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_30 (Conv2D)              │ (None, 248, 248, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_26 (MaxPooling2D) │ (None, 124, 124, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_31 (Conv2D)              │ (None, 122, 122, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_27 (MaxPooling2D) │ (None, 61, 61, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_32 (Conv2D)              │ (None, 59, 59, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_28 (MaxPooling2D) │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_10 (Flatten)            │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 3)              │       161,475 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 185,059 (722.89 KB)

 Trainable params: 185,059 (722.89 KB)

 Non-trainable params: 0 (0.00 B)

In [47]:
# Train the model
history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {accuracy}')

Epoch 1/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 39s 174ms/step - accuracy: 0.5763 - loss: 0.0146 - val_accuracy: 0.6214 - val_loss: 0.0049
Epoch 2/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 102ms/step - accuracy: 0.6188 - loss: 0.0048 - val_accuracy: 0.6214 - val_loss: 0.0049
Epoch 3/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 107ms/step - accuracy: 0.6248 - loss: 0.0046 - val_accuracy: 0.6314 - val_loss: 0.0049
Epoch 4/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 106ms/step - accuracy: 0.6314 - loss: 0.0045 - val_accuracy: 0.6179 - val_loss: 0.0048
Epoch 5/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 104ms/step - accuracy: 0.6425 - loss: 0.0045 - val_accuracy: 0.6321 - val_loss: 0.0047
Epoch 6/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 101ms/step - accuracy: 0.6628 - loss: 0.0043 - val_accuracy: 0.6464 - val_loss: 0.0046
Epoch 7/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 104ms/step - accuracy: 0.6753 - loss: 0.0042 - val_accuracy: 0.6357 - val_loss: 0.0046
Epoch 8/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 103ms/step - accuracy: 0.6957 - loss: 0

In [48]:
# Save the knowledge
model.save('m13_bill_length_head.keras')

# Forehead Color

In [49]:
# Keep only the appropriate values in each numpy array in the 'attributes' column
df['attributes_train'] = df['attributes'].apply(lambda x: x[152:167])

# Prepare the y
y = np.stack(df['attributes_train'].to_numpy(),axis=0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

length_of_output = y_test.shape[1]

In [50]:
# Define the CNN model - no color filter
inputs = Input(shape=(250, 250, 3))
x = Conv2D(16, (3, 3), activation='relu')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
outputs = Dense(length_of_output, activation='sigmoid')(x)  # Output layer with length_of_output units for the attributes

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "functional_23"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_11 (InputLayer)     │ (None, 250, 250, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_33 (Conv2D)              │ (None, 248, 248, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_29 (MaxPooling2D) │ (None, 124, 124, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_34 (Conv2D)              │ (None, 122, 122, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_30 (MaxPooling2D) │ (None, 61, 61, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_35 (Conv2D)              │ (None, 59, 59, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_31 (MaxPooling2D) │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_11 (Flatten)            │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 15)             │       807,375 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 830,959 (3.17 MB)

 Trainable params: 830,959 (3.17 MB)

 Non-trainable params: 0 (0.00 B)

In [51]:
# Train the model
history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {accuracy}')

Epoch 1/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 34s 166ms/step - accuracy: 0.3067 - loss: 0.0106 - val_accuracy: 0.3279 - val_loss: 0.0024
Epoch 2/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 105ms/step - accuracy: 0.3240 - loss: 0.0023 - val_accuracy: 0.3279 - val_loss: 0.0024
Epoch 3/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 105ms/step - accuracy: 0.3301 - loss: 0.0023 - val_accuracy: 0.3279 - val_loss: 0.0024
Epoch 4/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 106ms/step - accuracy: 0.3286 - loss: 0.0023 - val_accuracy: 0.3279 - val_loss: 0.0024
Epoch 5/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 107ms/step - accuracy: 0.3371 - loss: 0.0023 - val_accuracy: 0.3279 - val_loss: 0.0024
Epoch 6/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 103ms/step - accuracy: 0.3192 - loss: 0.0023 - val_accuracy: 0.3279 - val_loss: 0.0024
Epoch 7/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 104ms/step - accuracy: 0.3384 - loss: 0.0023 - val_accuracy: 0.3279 - val_loss: 0.0024
Epoch 8/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 102ms/step - accuracy: 0.3193 - loss: 0

In [52]:
# Save the knowledge
model.save('m14_forehead_color.keras')

# Under Tail Color

In [53]:
# Keep only the appropriate values in each numpy array in the 'attributes' column
df['attributes_train'] = df['attributes'].apply(lambda x: x[167:182])

# Prepare the y
y = np.stack(df['attributes_train'].to_numpy(),axis=0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

length_of_output = y_test.shape[1]

In [54]:
# Define the CNN model - no color filter
inputs = Input(shape=(250, 250, 3))
x = Conv2D(16, (3, 3), activation='relu')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
outputs = Dense(length_of_output, activation='sigmoid')(x)  # Output layer with length_of_output units for the attributes

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "functional_25"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_12 (InputLayer)     │ (None, 250, 250, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_36 (Conv2D)              │ (None, 248, 248, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_32 (MaxPooling2D) │ (None, 124, 124, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_37 (Conv2D)              │ (None, 122, 122, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_33 (MaxPooling2D) │ (None, 61, 61, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_38 (Conv2D)              │ (None, 59, 59, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_34 (MaxPooling2D) │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_12 (Flatten)            │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 15)             │       807,375 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 830,959 (3.17 MB)

 Trainable params: 830,959 (3.17 MB)

 Non-trainable params: 0 (0.00 B)

In [55]:
# Train the model
history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {accuracy}')

Epoch 1/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 30s 156ms/step - accuracy: 0.3120 - loss: 0.0102 - val_accuracy: 0.3479 - val_loss: 0.0026
Epoch 2/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 104ms/step - accuracy: 0.3406 - loss: 0.0026 - val_accuracy: 0.3479 - val_loss: 0.0026
Epoch 3/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 107ms/step - accuracy: 0.3343 - loss: 0.0026 - val_accuracy: 0.3479 - val_loss: 0.0026
Epoch 4/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 107ms/step - accuracy: 0.3387 - loss: 0.0026 - val_accuracy: 0.3479 - val_loss: 0.0026
Epoch 5/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 109ms/step - accuracy: 0.3423 - loss: 0.0026 - val_accuracy: 0.3479 - val_loss: 0.0026
Epoch 6/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 102ms/step - accuracy: 0.3435 - loss: 0.0026 - val_accuracy: 0.3479 - val_loss: 0.0026
Epoch 7/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 103ms/step - accuracy: 0.3409 - loss: 0.0026 - val_accuracy: 0.3479 - val_loss: 0.0026
Epoch 8/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 104ms/step - accuracy: 0.3410 - loss: 0

In [56]:
# Save the knowledge
model.save('m15_under_tail_color.keras')

# Nape Color

In [57]:
# Keep only the appropriate values in each numpy array in the 'attributes' column
df['attributes_train'] = df['attributes'].apply(lambda x: x[182:197])

# Prepare the y
y = np.stack(df['attributes_train'].to_numpy(),axis=0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

length_of_output = y_test.shape[1]

In [58]:
# Define the CNN model - no color filter
inputs = Input(shape=(250, 250, 3))
x = Conv2D(16, (3, 3), activation='relu')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
outputs = Dense(length_of_output, activation='sigmoid')(x)  # Output layer with length_of_output units for the attributes

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "functional_27"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_13 (InputLayer)     │ (None, 250, 250, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_39 (Conv2D)              │ (None, 248, 248, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_35 (MaxPooling2D) │ (None, 124, 124, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_40 (Conv2D)              │ (None, 122, 122, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_36 (MaxPooling2D) │ (None, 61, 61, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_41 (Conv2D)              │ (None, 59, 59, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_37 (MaxPooling2D) │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_13 (Flatten)            │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 15)             │       807,375 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 830,959 (3.17 MB)

 Trainable params: 830,959 (3.17 MB)

 Non-trainable params: 0 (0.00 B)

In [59]:
# Train the model
history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {accuracy}')

Epoch 1/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 36s 189ms/step - accuracy: 0.1986 - loss: 0.0103 - val_accuracy: 0.2329 - val_loss: 0.0025
Epoch 2/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 107ms/step - accuracy: 0.2243 - loss: 0.0025 - val_accuracy: 0.2329 - val_loss: 0.0025
Epoch 3/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 106ms/step - accuracy: 0.2191 - loss: 0.0025 - val_accuracy: 0.2329 - val_loss: 0.0025
Epoch 4/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 105ms/step - accuracy: 0.2183 - loss: 0.0025 - val_accuracy: 0.2329 - val_loss: 0.0025
Epoch 5/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 107ms/step - accuracy: 0.2295 - loss: 0.0025 - val_accuracy: 0.2329 - val_loss: 0.0025
Epoch 6/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 105ms/step - accuracy: 0.2219 - loss: 0.0025 - val_accuracy: 0.2329 - val_loss: 0.0025
Epoch 7/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 104ms/step - accuracy: 0.2197 - loss: 0.0025 - val_accuracy: 0.2329 - val_loss: 0.0025
Epoch 8/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 102ms/step - accuracy: 0.2175 - loss: 0

In [60]:
# Save the knowledge
model.save('m16_nape_color.keras')

# Belly Color

In [61]:
# Keep only the appropriate values in each numpy array in the 'attributes' column
df['attributes_train'] = df['attributes'].apply(lambda x: x[197:212])

# Prepare the y
y = np.stack(df['attributes_train'].to_numpy(),axis=0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

length_of_output = y_test.shape[1]

In [62]:
# Define the CNN model - no color filter
inputs = Input(shape=(250, 250, 3))
x = Conv2D(16, (3, 3), activation='relu')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
outputs = Dense(length_of_output, activation='sigmoid')(x)  # Output layer with length_of_output units for the attributes

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "functional_29"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_14 (InputLayer)     │ (None, 250, 250, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_42 (Conv2D)              │ (None, 248, 248, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_38 (MaxPooling2D) │ (None, 124, 124, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_43 (Conv2D)              │ (None, 122, 122, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_39 (MaxPooling2D) │ (None, 61, 61, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_44 (Conv2D)              │ (None, 59, 59, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_40 (MaxPooling2D) │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_14 (Flatten)            │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 15)             │       807,375 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 830,959 (3.17 MB)

 Trainable params: 830,959 (3.17 MB)

 Non-trainable params: 0 (0.00 B)

In [63]:
# Train the model
history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {accuracy}')

Epoch 1/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 34s 170ms/step - accuracy: 0.4101 - loss: 0.0111 - val_accuracy: 0.4014 - val_loss: 0.0027
Epoch 2/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 104ms/step - accuracy: 0.4343 - loss: 0.0027 - val_accuracy: 0.4014 - val_loss: 0.0027
Epoch 3/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 105ms/step - accuracy: 0.4241 - loss: 0.0027 - val_accuracy: 0.4014 - val_loss: 0.0027
Epoch 4/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 104ms/step - accuracy: 0.4237 - loss: 0.0027 - val_accuracy: 0.4014 - val_loss: 0.0027
Epoch 5/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 103ms/step - accuracy: 0.4384 - loss: 0.0027 - val_accuracy: 0.4014 - val_loss: 0.0027
Epoch 6/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 101ms/step - accuracy: 0.4261 - loss: 0.0027 - val_accuracy: 0.4014 - val_loss: 0.0027
Epoch 7/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 101ms/step - accuracy: 0.4288 - loss: 0.0027 - val_accuracy: 0.4014 - val_loss: 0.0027
Epoch 8/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 103ms/step - accuracy: 0.4162 - loss: 0

In [64]:
# Save the knowledge
model.save('m17_belly_color.keras')

# Wing Shape

In [65]:
# Keep only the appropriate values in each numpy array in the 'attributes' column
df['attributes_train'] = df['attributes'].apply(lambda x: x[212:217])

# Prepare the y
y = np.stack(df['attributes_train'].to_numpy(),axis=0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

length_of_output = y_test.shape[1]

In [66]:
# Define the CNN model - no color filter
inputs = Input(shape=(250, 250, 3))
x = Conv2D(16, (3, 3), activation='relu')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
outputs = Dense(length_of_output, activation='sigmoid')(x)  # Output layer with length_of_output units for the attributes

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "functional_31"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_15 (InputLayer)     │ (None, 250, 250, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_45 (Conv2D)              │ (None, 248, 248, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_41 (MaxPooling2D) │ (None, 124, 124, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_46 (Conv2D)              │ (None, 122, 122, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_42 (MaxPooling2D) │ (None, 61, 61, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_47 (Conv2D)              │ (None, 59, 59, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_43 (MaxPooling2D) │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_15 (Flatten)            │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 5)              │       269,125 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 292,709 (1.12 MB)

 Trainable params: 292,709 (1.12 MB)

 Non-trainable params: 0 (0.00 B)

In [67]:
# Train the model
history = model.fit(X_train, y_train, epochs=7, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {accuracy}')

Epoch 1/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 34s 172ms/step - accuracy: 0.6806 - loss: 0.0107 - val_accuracy: 0.7243 - val_loss: 0.0046
Epoch 2/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 105ms/step - accuracy: 0.7210 - loss: 0.0045 - val_accuracy: 0.7243 - val_loss: 0.0046
Epoch 3/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 107ms/step - accuracy: 0.7130 - loss: 0.0045 - val_accuracy: 0.7243 - val_loss: 0.0046
Epoch 4/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 106ms/step - accuracy: 0.7038 - loss: 0.0045 - val_accuracy: 0.7243 - val_loss: 0.0046
Epoch 5/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 104ms/step - accuracy: 0.7101 - loss: 0.0045 - val_accuracy: 0.7243 - val_loss: 0.0046
Epoch 6/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 103ms/step - accuracy: 0.7180 - loss: 0.0045 - val_accuracy: 0.7243 - val_loss: 0.0046
Epoch 7/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 101ms/step - accuracy: 0.7216 - loss: 0.0045 - val_accuracy: 0.7243 - val_loss: 0.0046
Epoch 8/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 103ms/step - accuracy: 0.7182 - loss: 0

In [68]:
# Save the knowledge
model.save('m18_wing_shape.keras')

# Size

In [69]:
# Keep only the appropriate values in each numpy array in the 'attributes' column
df['attributes_train'] = df['attributes'].apply(lambda x: x[217:222])

# Prepare the y
y = np.stack(df['attributes_train'].to_numpy(),axis=0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

length_of_output = y_test.shape[1]

In [70]:
# Define the CNN model - no color filter
inputs = Input(shape=(250, 250, 3))
x = Conv2D(16, (3, 3), activation='relu')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
outputs = Dense(length_of_output, activation='sigmoid')(x)  # Output layer with length_of_output units for the attributes

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "functional_33"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_16 (InputLayer)     │ (None, 250, 250, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_48 (Conv2D)              │ (None, 248, 248, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_44 (MaxPooling2D) │ (None, 124, 124, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_49 (Conv2D)              │ (None, 122, 122, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_45 (MaxPooling2D) │ (None, 61, 61, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_50 (Conv2D)              │ (None, 59, 59, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_46 (MaxPooling2D) │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_16 (Flatten)            │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 5)              │       269,125 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 292,709 (1.12 MB)

 Trainable params: 292,709 (1.12 MB)

 Non-trainable params: 0 (0.00 B)

In [71]:
# Train the model
history = model.fit(X_train, y_train, epochs=7, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {accuracy}')

Epoch 1/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 32s 162ms/step - accuracy: 0.6468 - loss: 0.0127 - val_accuracy: 0.6821 - val_loss: 0.0062
Epoch 2/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 104ms/step - accuracy: 0.6928 - loss: 0.0062 - val_accuracy: 0.6821 - val_loss: 0.0062
Epoch 3/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 103ms/step - accuracy: 0.6892 - loss: 0.0061 - val_accuracy: 0.6821 - val_loss: 0.0062
Epoch 4/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 21s 121ms/step - accuracy: 0.6725 - loss: 0.0062 - val_accuracy: 0.6821 - val_loss: 0.0062
Epoch 5/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 106ms/step - accuracy: 0.6814 - loss: 0.0062 - val_accuracy: 0.6821 - val_loss: 0.0062
Epoch 6/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 105ms/step - accuracy: 0.6833 - loss: 0.0062 - val_accuracy: 0.6821 - val_loss: 0.0062
Epoch 7/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 102ms/step - accuracy: 0.6759 - loss: 0.0062 - val_accuracy: 0.6821 - val_loss: 0.0062
Epoch 8/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 104ms/step - accuracy: 0.6867 - loss: 0

In [72]:
# Save the knowledge
model.save('m19_size.keras')

# Shape

In [73]:
# Keep only the appropriate values in each numpy array in the 'attributes' column
df['attributes_train'] = df['attributes'].apply(lambda x: x[222:236])

# Prepare the y
y = np.stack(df['attributes_train'].to_numpy(),axis=0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

length_of_output = y_test.shape[1]

In [74]:
# Define the CNN model - no color filter
inputs = Input(shape=(250, 250, 3))
x = Conv2D(16, (3, 3), activation='relu')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
outputs = Dense(length_of_output, activation='sigmoid')(x)  # Output layer with length_of_output units for the attributes

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "functional_35"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_17 (InputLayer)     │ (None, 250, 250, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_51 (Conv2D)              │ (None, 248, 248, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_47 (MaxPooling2D) │ (None, 124, 124, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_52 (Conv2D)              │ (None, 122, 122, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_48 (MaxPooling2D) │ (None, 61, 61, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_53 (Conv2D)              │ (None, 59, 59, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_49 (MaxPooling2D) │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_17 (Flatten)            │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 14)             │       753,550 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 777,134 (2.96 MB)

 Trainable params: 777,134 (2.96 MB)

 Non-trainable params: 0 (0.00 B)

In [75]:
# Train the model
history = model.fit(X_train, y_train, epochs=7, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {accuracy}')

Epoch 1/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 33s 169ms/step - accuracy: 0.6514 - loss: 0.0106 - val_accuracy: 0.7036 - val_loss: 0.0020
Epoch 2/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 107ms/step - accuracy: 0.7070 - loss: 0.0020 - val_accuracy: 0.7036 - val_loss: 0.0020
Epoch 3/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 108ms/step - accuracy: 0.6866 - loss: 0.0020 - val_accuracy: 0.7036 - val_loss: 0.0020
Epoch 4/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 108ms/step - accuracy: 0.7139 - loss: 0.0020 - val_accuracy: 0.7036 - val_loss: 0.0020
Epoch 5/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 109ms/step - accuracy: 0.7081 - loss: 0.0020 - val_accuracy: 0.7036 - val_loss: 0.0020
Epoch 6/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 105ms/step - accuracy: 0.7051 - loss: 0.0020 - val_accuracy: 0.7036 - val_loss: 0.0020
Epoch 7/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 104ms/step - accuracy: 0.6997 - loss: 0.0020 - val_accuracy: 0.7036 - val_loss: 0.0020
Epoch 8/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 105ms/step - accuracy: 0.7011 - loss: 0

In [76]:
# Save the knowledge
model.save('m20_shape.keras')

# Back Pattern

In [77]:
# Keep only the appropriate values in each numpy array in the 'attributes' column
df['attributes_train'] = df['attributes'].apply(lambda x: x[236:240])

# Prepare the y
y = np.stack(df['attributes_train'].to_numpy(),axis=0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

length_of_output = y_test.shape[1]

In [78]:
# Define the CNN model - no color filter
inputs = Input(shape=(250, 250, 3))
x = Conv2D(16, (3, 3), activation='relu')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
outputs = Dense(length_of_output, activation='sigmoid')(x)  # Output layer with length_of_output units for the attributes

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "functional_37"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_18 (InputLayer)     │ (None, 250, 250, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_54 (Conv2D)              │ (None, 248, 248, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_50 (MaxPooling2D) │ (None, 124, 124, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_55 (Conv2D)              │ (None, 122, 122, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_51 (MaxPooling2D) │ (None, 61, 61, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_56 (Conv2D)              │ (None, 59, 59, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_52 (MaxPooling2D) │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_18 (Flatten)            │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 4)              │       215,300 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 238,884 (933.14 KB)

 Trainable params: 238,884 (933.14 KB)

 Non-trainable params: 0 (0.00 B)

In [79]:
# Train the model
history = model.fit(X_train, y_train, epochs=7, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {accuracy}')

Epoch 1/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 35s 180ms/step - accuracy: 0.5434 - loss: 0.0129 - val_accuracy: 0.2629 - val_loss: 0.0070
Epoch 2/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 107ms/step - accuracy: 0.2545 - loss: 0.0069 - val_accuracy: 0.2871 - val_loss: 0.0070
Epoch 3/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 105ms/step - accuracy: 0.2957 - loss: 0.0069 - val_accuracy: 0.3329 - val_loss: 0.0070
Epoch 4/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 105ms/step - accuracy: 0.3150 - loss: 0.0069 - val_accuracy: 0.4000 - val_loss: 0.0070
Epoch 5/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 106ms/step - accuracy: 0.3434 - loss: 0.0069 - val_accuracy: 0.5557 - val_loss: 0.0070
Epoch 6/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 107ms/step - accuracy: 0.5070 - loss: 0.0069 - val_accuracy: 0.6321 - val_loss: 0.0070
Epoch 7/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 101ms/step - accuracy: 0.4974 - loss: 0.0069 - val_accuracy: 0.3400 - val_loss: 0.0070
Epoch 8/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 104ms/step - accuracy: 0.3609 - loss: 0

In [80]:
# Save the knowledge
model.save('m21_back_pattern.keras')

# Tail Pattern

In [81]:
# Keep only the appropriate values in each numpy array in the 'attributes' column
df['attributes_train'] = df['attributes'].apply(lambda x: x[240:244])

# Prepare the y
y = np.stack(df['attributes_train'].to_numpy(),axis=0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

length_of_output = y_test.shape[1]

In [82]:
# Define the CNN model - no color filter
inputs = Input(shape=(250, 250, 3))
x = Conv2D(16, (3, 3), activation='relu')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
outputs = Dense(length_of_output, activation='sigmoid')(x)  # Output layer with length_of_output units for the attributes

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "functional_39"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_19 (InputLayer)     │ (None, 250, 250, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_57 (Conv2D)              │ (None, 248, 248, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_53 (MaxPooling2D) │ (None, 124, 124, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_58 (Conv2D)              │ (None, 122, 122, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_54 (MaxPooling2D) │ (None, 61, 61, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_59 (Conv2D)              │ (None, 59, 59, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_55 (MaxPooling2D) │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_19 (Flatten)            │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 4)              │       215,300 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 238,884 (933.14 KB)

 Trainable params: 238,884 (933.14 KB)

 Non-trainable params: 0 (0.00 B)

In [83]:
# Train the model
history = model.fit(X_train, y_train, epochs=7, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {accuracy}')

Epoch 1/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 31s 159ms/step - accuracy: 0.5417 - loss: 0.0136 - val_accuracy: 0.5829 - val_loss: 0.0068
Epoch 2/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 105ms/step - accuracy: 0.4815 - loss: 0.0074 - val_accuracy: 0.4886 - val_loss: 0.0068
Epoch 3/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 107ms/step - accuracy: 0.4940 - loss: 0.0066 - val_accuracy: 0.4886 - val_loss: 0.0068
Epoch 4/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 106ms/step - accuracy: 0.4994 - loss: 0.0067 - val_accuracy: 0.4886 - val_loss: 0.0068
Epoch 5/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 107ms/step - accuracy: 0.5052 - loss: 0.0066 - val_accuracy: 0.4886 - val_loss: 0.0068
Epoch 6/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 20s 112ms/step - accuracy: 0.4914 - loss: 0.0067 - val_accuracy: 0.4886 - val_loss: 0.0068
Epoch 7/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 103ms/step - accuracy: 0.4929 - loss: 0.0066 - val_accuracy: 0.4886 - val_loss: 0.0068
Epoch 8/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 104ms/step - accuracy: 0.5072 - loss: 0

In [84]:
# Save the knowledge
model.save('m22_tail_pattern.keras')

# Belly Pattern

In [85]:
# Keep only the appropriate values in each numpy array in the 'attributes' column
df['attributes_train'] = df['attributes'].apply(lambda x: x[244:248])

# Prepare the y
y = np.stack(df['attributes_train'].to_numpy(),axis=0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

length_of_output = y_test.shape[1]

In [86]:
# Define the CNN model - no color filter
inputs = Input(shape=(250, 250, 3))
x = Conv2D(16, (3, 3), activation='relu')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
outputs = Dense(length_of_output, activation='sigmoid')(x)  # Output layer with length_of_output units for the attributes

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "functional_41"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_20 (InputLayer)     │ (None, 250, 250, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_60 (Conv2D)              │ (None, 248, 248, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_56 (MaxPooling2D) │ (None, 124, 124, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_61 (Conv2D)              │ (None, 122, 122, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_57 (MaxPooling2D) │ (None, 61, 61, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_62 (Conv2D)              │ (None, 59, 59, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_58 (MaxPooling2D) │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_20 (Flatten)            │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 4)              │       215,300 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 238,884 (933.14 KB)

 Trainable params: 238,884 (933.14 KB)

 Non-trainable params: 0 (0.00 B)

In [87]:
# Train the model
history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {accuracy}')

Epoch 1/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 28s 148ms/step - accuracy: 0.7483 - loss: 0.0153 - val_accuracy: 0.8064 - val_loss: 0.0088
Epoch 2/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 105ms/step - accuracy: 0.8045 - loss: 0.0087 - val_accuracy: 0.8064 - val_loss: 0.0088
Epoch 3/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 105ms/step - accuracy: 0.8015 - loss: 0.0087 - val_accuracy: 0.8064 - val_loss: 0.0088
Epoch 4/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 108ms/step - accuracy: 0.7905 - loss: 0.0086 - val_accuracy: 0.8064 - val_loss: 0.0088
Epoch 5/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 109ms/step - accuracy: 0.8011 - loss: 0.0087 - val_accuracy: 0.8064 - val_loss: 0.0088
Epoch 6/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 104ms/step - accuracy: 0.7985 - loss: 0.0087 - val_accuracy: 0.8064 - val_loss: 0.0088
Epoch 7/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 103ms/step - accuracy: 0.7965 - loss: 0.0087 - val_accuracy: 0.8064 - val_loss: 0.0088
Epoch 8/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 101ms/step - accuracy: 0.7912 - loss: 0

In [88]:
# Save the knowledge
model.save('m23_belly_pattern.keras')

# Primary Color

In [89]:
# Keep only the appropriate values in each numpy array in the 'attributes' column
df['attributes_train'] = df['attributes'].apply(lambda x: x[248:263])

# Prepare the y
y = np.stack(df['attributes_train'].to_numpy(),axis=0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

length_of_output = y_test.shape[1]

In [90]:
from tensorflow.keras.layers import AveragePooling2D

# Define the CNN model with a blurring layer
inputs = Input(shape=(250, 250, 3))
blurred = AveragePooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(inputs)  # Blurring layer
x = Conv2D(16, (3, 3), activation='relu')(blurred)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
outputs = Dense(length_of_output, activation='sigmoid')(x)  # Output layer with length_of_output units for the attributes

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Summary of the model
model.summary()


Model: "functional_43"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_21 (InputLayer)     │ (None, 250, 250, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (None, 250, 250, 3)    │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_63 (Conv2D)              │ (None, 248, 248, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_59 (MaxPooling2D) │ (None, 124, 124, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_64 (Conv2D)              │ (None, 122, 122, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_60 (MaxPooling2D) │ (None, 61, 61, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_65 (Conv2D)              │ (None, 59, 59, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_61 (MaxPooling2D) │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_21 (Flatten)            │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_21 (Dropout)            │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 15)             │       807,375 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 830,959 (3.17 MB)

 Trainable params: 830,959 (3.17 MB)

 Non-trainable params: 0 (0.00 B)

In [91]:
# Train the model
history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {accuracy}')

Epoch 1/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 31s 158ms/step - accuracy: 0.1826 - loss: 0.0103 - val_accuracy: 0.2043 - val_loss: 0.0033
Epoch 2/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 106ms/step - accuracy: 0.1963 - loss: 0.0033 - val_accuracy: 0.2043 - val_loss: 0.0033
Epoch 3/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 106ms/step - accuracy: 0.1799 - loss: 0.0033 - val_accuracy: 0.2043 - val_loss: 0.0033
Epoch 4/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 107ms/step - accuracy: 0.1866 - loss: 0.0033 - val_accuracy: 0.2043 - val_loss: 0.0033
Epoch 5/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 107ms/step - accuracy: 0.1879 - loss: 0.0033 - val_accuracy: 0.2043 - val_loss: 0.0033
Epoch 6/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 108ms/step - accuracy: 0.1926 - loss: 0.0033 - val_accuracy: 0.2036 - val_loss: 0.0033
Epoch 7/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 103ms/step - accuracy: 0.1805 - loss: 0.0033 - val_accuracy: 0.2036 - val_loss: 0.0033
Epoch 8/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 104ms/step - accuracy: 0.2009 - loss: 0

In [92]:
# Save the knowledge
model.save('m24_primary_color.keras')

# Leg Color

In [93]:
# Keep only the appropriate values in each numpy array in the 'attributes' column
df['attributes_train'] = df['attributes'].apply(lambda x: x[263:278])

# Prepare the y
y = np.stack(df['attributes_train'].to_numpy(),axis=0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

length_of_output = y_test.shape[1]

In [94]:
# Define the CNN model with a blurring layer
inputs = Input(shape=(250, 250, 3))
blurred = AveragePooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(inputs)  # Blurring layer
x = Conv2D(16, (3, 3), activation='relu')(blurred)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
outputs = Dense(length_of_output, activation='sigmoid')(x)  # Output layer with length_of_output units for the attributes

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "functional_45"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_22 (InputLayer)     │ (None, 250, 250, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_1             │ (None, 250, 250, 3)    │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_66 (Conv2D)              │ (None, 248, 248, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_62 (MaxPooling2D) │ (None, 124, 124, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_67 (Conv2D)              │ (None, 122, 122, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_63 (MaxPooling2D) │ (None, 61, 61, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_68 (Conv2D)              │ (None, 59, 59, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_64 (MaxPooling2D) │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_22 (Flatten)            │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 15)             │       807,375 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 830,959 (3.17 MB)

 Trainable params: 830,959 (3.17 MB)

 Non-trainable params: 0 (0.00 B)

In [95]:
# Train the model
history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {accuracy}')

Epoch 1/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 31s 163ms/step - accuracy: 0.2952 - loss: 0.0103 - val_accuracy: 0.3164 - val_loss: 0.0017
Epoch 2/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 105ms/step - accuracy: 0.3433 - loss: 0.0017 - val_accuracy: 0.3164 - val_loss: 0.0017
Epoch 3/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 107ms/step - accuracy: 0.3250 - loss: 0.0017 - val_accuracy: 0.3164 - val_loss: 0.0017
Epoch 4/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 107ms/step - accuracy: 0.3247 - loss: 0.0017 - val_accuracy: 0.3164 - val_loss: 0.0017
Epoch 5/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 106ms/step - accuracy: 0.3190 - loss: 0.0017 - val_accuracy: 0.3164 - val_loss: 0.0017
Epoch 6/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 109ms/step - accuracy: 0.3201 - loss: 0.0017 - val_accuracy: 0.3164 - val_loss: 0.0017
Epoch 7/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 109ms/step - accuracy: 0.3230 - loss: 0.0017 - val_accuracy: 0.3164 - val_loss: 0.0017
Epoch 8/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 105ms/step - accuracy: 0.3178 - loss: 0

In [96]:
# Save the knowledge
model.save('m25_leg_color.keras')

# Bill Color

In [97]:
# Keep only the appropriate values in each numpy array in the 'attributes' column
df['attributes_train'] = df['attributes'].apply(lambda x: x[278:293])

# Prepare the y
y = np.stack(df['attributes_train'].to_numpy(),axis=0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

length_of_output = y_test.shape[1]

In [98]:
# Define the CNN model - no color filter
inputs = Input(shape=(250, 250, 3))
x = Conv2D(16, (3, 3), activation='relu')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
outputs = Dense(length_of_output, activation='sigmoid')(x)  # Output layer with length_of_output units for the attributes

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "functional_47"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_23 (InputLayer)     │ (None, 250, 250, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_69 (Conv2D)              │ (None, 248, 248, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_65 (MaxPooling2D) │ (None, 124, 124, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_70 (Conv2D)              │ (None, 122, 122, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_66 (MaxPooling2D) │ (None, 61, 61, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_71 (Conv2D)              │ (None, 59, 59, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_67 (MaxPooling2D) │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_23 (Flatten)            │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_23 (Dropout)            │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 15)             │       807,375 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 830,959 (3.17 MB)

 Trainable params: 830,959 (3.17 MB)

 Non-trainable params: 0 (0.00 B)

In [99]:
# Train the model
history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {accuracy}')

Epoch 1/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 38s 200ms/step - accuracy: 0.5300 - loss: 0.0098 - val_accuracy: 0.5671 - val_loss: 0.0020
Epoch 2/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 108ms/step - accuracy: 0.5473 - loss: 0.0020 - val_accuracy: 0.5671 - val_loss: 0.0020
Epoch 3/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 109ms/step - accuracy: 0.5586 - loss: 0.0020 - val_accuracy: 0.5671 - val_loss: 0.0020
Epoch 4/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 107ms/step - accuracy: 0.5620 - loss: 0.0020 - val_accuracy: 0.5671 - val_loss: 0.0020
Epoch 5/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 106ms/step - accuracy: 0.5465 - loss: 0.0020 - val_accuracy: 0.5671 - val_loss: 0.0020
Epoch 6/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 105ms/step - accuracy: 0.5558 - loss: 0.0020 - val_accuracy: 0.5671 - val_loss: 0.0020
Epoch 7/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 104ms/step - accuracy: 0.5548 - loss: 0.0020 - val_accuracy: 0.5671 - val_loss: 0.0020
Epoch 8/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 106ms/step - accuracy: 0.5533 - loss: 0

In [100]:
# Save the knowledge
model.save('m26_bill_color.keras')

# Crown Color

In [101]:
# Keep only the appropriate values in each numpy array in the 'attributes' column
df['attributes_train'] = df['attributes'].apply(lambda x: x[293:308])

# Prepare the y
y = np.stack(df['attributes_train'].to_numpy(),axis=0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

length_of_output = y_test.shape[1]

In [102]:
# Define the CNN model - no color filter
inputs = Input(shape=(250, 250, 3))
x = Conv2D(16, (3, 3), activation='relu')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
outputs = Dense(length_of_output, activation='sigmoid')(x)  # Output layer with length_of_output units for the attributes

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "functional_49"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_24 (InputLayer)     │ (None, 250, 250, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_72 (Conv2D)              │ (None, 248, 248, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_68 (MaxPooling2D) │ (None, 124, 124, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_73 (Conv2D)              │ (None, 122, 122, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_69 (MaxPooling2D) │ (None, 61, 61, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_74 (Conv2D)              │ (None, 59, 59, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_70 (MaxPooling2D) │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_24 (Flatten)            │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 15)             │       807,375 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 830,959 (3.17 MB)

 Trainable params: 830,959 (3.17 MB)

 Non-trainable params: 0 (0.00 B)

In [103]:
# Train the model
history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {accuracy}')

Epoch 1/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 33s 168ms/step - accuracy: 0.2914 - loss: 0.0110 - val_accuracy: 0.2821 - val_loss: 0.0024
Epoch 2/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 107ms/step - accuracy: 0.2684 - loss: 0.0024 - val_accuracy: 0.2821 - val_loss: 0.0024
Epoch 3/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 110ms/step - accuracy: 0.2707 - loss: 0.0024 - val_accuracy: 0.2821 - val_loss: 0.0024
Epoch 4/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 110ms/step - accuracy: 0.2692 - loss: 0.0024 - val_accuracy: 0.2821 - val_loss: 0.0024
Epoch 5/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 108ms/step - accuracy: 0.2737 - loss: 0.0024 - val_accuracy: 0.2821 - val_loss: 0.0024
Epoch 6/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 107ms/step - accuracy: 0.2763 - loss: 0.0024 - val_accuracy: 0.2821 - val_loss: 0.0024
Epoch 7/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 104ms/step - accuracy: 0.2677 - loss: 0.0024 - val_accuracy: 0.2821 - val_loss: 0.0024
Epoch 8/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 103ms/step - accuracy: 0.2648 - loss: 0

In [104]:
# Save the knowledge
model.save('m27_crown_color.keras')

# Wing Pattern

In [105]:
# Keep only the appropriate values in each numpy array in the 'attributes' column
df['attributes_train'] = df['attributes'].apply(lambda x: x[308:312])

# Prepare the y
y = np.stack(df['attributes_train'].to_numpy(),axis=0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

length_of_output = y_test.shape[1]

In [106]:
# Define the CNN model - no color filter
inputs = Input(shape=(250, 250, 3))
x = Conv2D(16, (3, 3), activation='relu')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
outputs = Dense(length_of_output, activation='sigmoid')(x)  # Output layer with length_of_output units for the attributes

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "functional_51"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_25 (InputLayer)     │ (None, 250, 250, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_75 (Conv2D)              │ (None, 248, 248, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_71 (MaxPooling2D) │ (None, 124, 124, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_76 (Conv2D)              │ (None, 122, 122, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_72 (MaxPooling2D) │ (None, 61, 61, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_77 (Conv2D)              │ (None, 59, 59, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_73 (MaxPooling2D) │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_25 (Flatten)            │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 4)              │       215,300 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 238,884 (933.14 KB)

 Trainable params: 238,884 (933.14 KB)

 Non-trainable params: 0 (0.00 B)

In [107]:
# Train the model
history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {accuracy}')

Epoch 1/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 37s 188ms/step - accuracy: 0.3336 - loss: 0.0126 - val_accuracy: 0.3621 - val_loss: 0.0068
Epoch 2/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 105ms/step - accuracy: 0.3405 - loss: 0.0068 - val_accuracy: 0.3621 - val_loss: 0.0068
Epoch 3/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 110ms/step - accuracy: 0.3488 - loss: 0.0068 - val_accuracy: 0.3621 - val_loss: 0.0068
Epoch 4/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 106ms/step - accuracy: 0.3493 - loss: 0.0067 - val_accuracy: 0.3621 - val_loss: 0.0068
Epoch 5/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 19s 107ms/step - accuracy: 0.3516 - loss: 0.0068 - val_accuracy: 0.3621 - val_loss: 0.0068
Epoch 6/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 103ms/step - accuracy: 0.3509 - loss: 0.0067 - val_accuracy: 0.3621 - val_loss: 0.0068
Epoch 7/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 103ms/step - accuracy: 0.3408 - loss: 0.0068 - val_accuracy: 0.3621 - val_loss: 0.0068
Epoch 8/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 18s 102ms/step - accuracy: 0.3635 - loss: 0

In [108]:
# Save the knowledge
model.save('m28_wing_pattern.keras')

# Ensemble tests & predictions

In [ ]:
#prepare data
df = pd.read_csv('test_images_path.csv')
df['image_path'] = df['image_path'].str.lstrip('/')

def preprocess_image(image_path):
    image = Image.open(image_path)
    padded_image = ImageOps.pad(image, (500, 500), color='black')
    resized_image = padded_image.resize((250, 250), Image.LANCZOS)
#     return resized_image
    image_array = np.array(resized_image) / 255.0  # Normalize pixel values to [0, 1]
    if image_array.shape == (250, 250, 3):
        return image_array
    else:
        return np.stack((image_array,)*3, axis=-1)
#         return np.zeros((250, 250, 3))  # Return a blank image if the shape is incorrect
# Check if any entries in the 'processed_image' column are of type string

df['processed_image'] = df['image_path'].apply(preprocess_image)
X = np.stack(df['processed_image'].to_numpy(),axis=0)

In [ ]:
# efficientnet prediction
from tensorflow.keras.models import load_model
efnet = load_model('efnet.keras')
efnet_pred = efnet.predict(X)

In [ ]:
# ensemble test and predictions
import os
import re

# Get the list of all files in the working directory
all_files = os.listdir('.')

# Filter out the model files
model_files = [f for f in all_files if f.startswith('m') and f.endswith('.keras')]

# Custom sorting function to extract the numerical part of the filenames
def extract_number(filename):
    match = re.search(r'm(\d+)_', filename)
    return int(match.group(1)) if match else 0

# Sort the model files using the custom sorting function
model_files.sort(key=extract_number)

print(model_files)

In [ ]:
from tensorflow.keras.models import load_model
y=[]

for m in model_files:
    model = load_model(m)
    y.append(model.predict(X))


In [ ]:
combined_predictions = np.hstack(y)

class_names = np.load("class_names.npy", allow_pickle=True).item()
attributes = np.load("attributes.npy")
labels = class_labels = list(class_names.values())

In [ ]:
# try to ensemble
# from sklearn.ensemble import BaggingRegressor
# from sklearn.linear_model import LinearRegression

# model = BaggingRegressor(base_estimator=LinearRegression(), n_estimators=10, random_state=42)
# model.fit(combined_predictions, attributes)

# # Make predictions
# final_predictions = model.predict(combined_predictions)

In [ ]:
predictions = combined_predictions

# Function to find the closest label for a single prediction
def find_closest_label(predicted_attributes, attributes, labels):
    distances = np.linalg.norm(attributes - predicted_attributes, axis=1)
    closest_label_index = np.argmin(distances)
    return labels[closest_label_index]

results = []

# find the closest label for each prediction
for i, predicted_attributes in enumerate(predictions):
    closest_label = find_closest_label(predicted_attributes, attributes, labels)
    results.append({'id': i + 1, 'label': closest_label})

df = pd.DataFrame(results)
df.to_csv('predictions.csv',index=False)